# Problem 2: Political Network Connections

In this problem, you will analyze the network connections and strength between all persons and organizations in the *Trump World* using a combination of hash tables (i.e., dictionaries) and pandas dataframe.  

## The dataset

The dataset for this problem is built from public records, news reports, and other sources on the Trump family, his Cabinet picks, and top advisers - more than 1,500 people and organizations altogether. 

Each row represents a connection between a person and an organization (e.g., The Trump Organization Inc. and Donald J. Trump), a person and another person (e.g., Donald J. Trump and Linda McMahon), or two organizations (e.g., Bedford Hills Corp. and Seven Springs LLC).

Source: https://www.buzzfeednews.com/article/johntemplon/help-us-map-trumpworld

Before starting, please run the following cell to set up the environment and import the data to `Network`.

In [1]:
import math
import pandas as pd
import numpy as np
from collections import defaultdict
Network = pd.read_csv("../resource/asnlib/publicdata/network.csv",encoding='latin-1' )
assert len(Network) == 3380
Network.head()

,Entity A Type,Entity A,Entity B Type,Entity B,Connection_Strength
0,Organization,4 SHADOW TREE LANE MEMBER CORP.,Organization,4 SHADOW TREE LANE LLC,0.469155
1,Organization,40 WALL DEVELOPMENT ASSOCIATES LLC,Organization,40 WALL STREET LLC,0.035480
2,Organization,40 WALL STREET LLC,Organization,40 WALL STREET COMMERCIAL LLC,0.177874
3,Organization,40 WALL STREET MEMBER CORP.,Organization,40 WALL STREET LLC,0.236508
4,Organization,401 MEZZ VENTURE LLC,Organization,401 NORTH WABASH VENTURE LLC,0.169532


**Exercise 0** (1 points). Create a subset of the data frame named `Network_sub`, keeping only records where `Entity B` contains the keyword "TRUMP" (not case sensitive).

In [2]:
# Store the subset in Network_sub

Network_sub=Network[Network['Entity B'].str.contains('TRUMP|Trump|trump')]
display(Network_sub)

,Entity A Type,Entity A,Entity B Type,Entity B,Connection_Strength
5,Organization,401 MEZZ VENTURE LLC,Organization,TRUMP CHICAGO RETAIL LLC,0.852985
6,Organization,401 NORTH WABASH VENTURE LLC,Organization,TRUMP COMMERCIAL CHICAGO LLC,0.207837
7,Organization,401 NORTH WABASH VENTURE LLC,Organization,TRUMP PAYROLL CHICAGO LLC,0.910172
14,Person,ABE WALLACH,Organization,"THE TRUMP ORGANIZATION, INC.",0.452901
17,Organization,ACE ENTERTAINMENT HOLDINGS INC,Organization,TRUMP CASINOS INC.,0.202485
18,Organization,ACE ENTERTAINMENT HOLDINGS INC,Organization,TRUMP TAJ MAHAL INC.,0.487848
20,Organization,AETOS SA,Organization,TRUMP MARKS PUNTA DEL ESTE LLC,0.516288
25,Person,ALAN GARTEN,Organization,"THE TRUMP ORGANIZATION, INC.",0.569394
49,Person,ALLEN WEISSELBERG,Organization,THE DONALD J. TRUMP FOUNDATION,0.968860
50,Person,ALLEN WEISSELBERG,Organization,THE DONALD J. TRUMP REVOCABLE TRUST,0.528962


In [3]:
# Test cell: `test_subset`

assert type(Network_sub)==pd.DataFrame, "Your subset is not a panda dataframe"
assert list(Network_sub)==['Entity A Type','Entity A','Entity B Type','Entity B','Connection_Strength'], "Your subset columns are not consistent with the master dataset"
assert len(Network_sub)==648, "The length of your subset is not correct"

test = Network_sub.sort_values(by='Connection_Strength')
test.reset_index(drop=True, inplace=True)
assert test.loc[0,'Connection_Strength']==0.001315204
assert test.loc[200,'Connection_Strength']==0.312599997
assert test.loc[400,'Connection_Strength']==0.610184514
assert test.loc[647,'Connection_Strength']==0.996641965

print("\n(Passed.)")


(Passed.)


Now, let's take a look at part of the `Network_sub` data.

In [4]:
Network_sub[25:36]

,Entity A Type,Entity A,Entity B Type,Entity B,Connection_Strength
232,Person,BRIAN BAUDREAU,Organization,"THE TRUMP ORGANIZATION, INC.",0.249506
237,Organization,"BRIARCLIFF PROPERTIES, INC.",Organization,TRUMP BRIARCLIFF MANOR DEVELOPMENT LLC,0.102998
238,Person,BRITTANY HEBERT,Organization,THE ERIC TRUMP FOUNDATION,0.724913
257,Person,CARTER PAGE,Person,DONALD J. TRUMP,0.694884
280,Person,CHARLES P. REISS,Organization,"THE TRUMP ORGANIZATION, INC.",0.937458
283,Person,CHEN SITING AKA CHARLYNE CHEN,Organization,TRUMP ORGANIZATION LLC,0.137199
284,Organization,"CHEVY CHASE TRUST HOLDINGS, INC.",Organization,TRUMP NATIONAL GOLF CLUB WASHINGTON DC LLC,0.925422
286,Person,CHLOE MURDOCH,Person,IVANKA TRUMP,0.805567
294,Person,CHRISTL MAHFOUZ,Organization,THE ERIC TRUMP FOUNDATION,0.426780
326,Organization,DAEWOO AMERICA DEVELOPMENT (NEW YORK) CORP,Organization,TRUMP KOREA LLC,0.994785


**Exercise 1** (4 points). Write a function 

```python
def Connection_Strength(Network_sub, Entity_B_Type)
```

that takes two inputs

1. `Network_sub` is the dataset you get from exercise 0
2. `Entity_B_Type` can take two values: either `Person` or `Organization`

and for every entity A that is connected to entity B, based on the type of entity B, returns a nested dictionary (i.e. dictionary of dictionaries) of the form:

```python 
{Entity A: {Entity B: Connection_Strength, Entity B: Connection_Strength}, ... }```

For example: for entity A that is connected to entity B of type person, the function will return something like the following: 

```python
{'DONALD J. TRUMP': {'DONALD TRUMP JR.': 0.453990548,
  'ERIC TRUMP': 0.468002101,
  'IVANKA TRUMP': 0.773874808,
  'MARYANNE TRUMP BARRY': 0.330120053,
  'MELANIA TRUMP': 0.5171444000000001},
 'DONALD J. TRUMP FOR PRESIDENT, INC.': {'DONALD J. TRUMP': 0.377887355},
 'DONALD TRUMP JR.': {'ERIC TRUMP': 0.405052388, 'VANESSA TRUMP': 0.025756815},
 'GRACE MURDOCH': {'IVANKA TRUMP': 0.966637541},
 'IVANKA M. TRUMP BUSINESS TRUST': {'IVANKA TRUMP': 0.141785871}, ...}```

In [5]:
#display(Network_sub)

# https://stackoverflow.com/questions/16333296/how-do-you-create-nested-dict-in-python

def Connection_Strength(Network_sub, Entity_B_Type):
    assert type(Entity_B_Type) == str
    assert Entity_B_Type in ['Person', 'Organization']
    
    #Create subset based on Entity_B_Type
    sub=Network_sub[Network_sub['Entity B Type']==Entity_B_Type]
    
    #Create subset with relevant columns
    sub=sub[['Entity A','Entity B', 'Connection_Strength']]
    
    #Set index to Entity A
    sub.set_index(keys='Entity A', drop=True, inplace=True)
    display(sub)

    from collections import defaultdict
    dd=defaultdict(dict)
    
    for row in sub.itertuples():
        dd[row[0]][row[1]]=row[2]

    return dd


In [6]:
# Test Cell: `Connection_Strength`

# Create a dictonary 'Person' for entity B of type person
Person = Connection_Strength(Network_sub, 'Person')
# Create a dictionary 'Organization' for entity B of type organization
Organization = Connection_Strength(Network_sub, 'Organization')

assert type(Person)==dict or defaultdict, "Your function does not return a dictionary"
assert len(Person)==17, "Your result is wrong for entity B of type person"
assert len(Organization)==296, "Your result is wrong for entity B of type organization"

assert Person['DONALD J. TRUMP']=={'DONALD TRUMP JR.': 0.453990548,'ERIC TRUMP': 0.468002101,'IVANKA TRUMP': 0.773874808,
  'MARYANNE TRUMP BARRY': 0.330120053,'MELANIA TRUMP': 0.5171444000000001}, "Wrong result"
assert Person['DONALD J. TRUMP FOR PRESIDENT, INC.']=={'DONALD J. TRUMP': 0.377887355}, "Wrong result"
assert Person['WENDI DENG MURDOCH']=={'IVANKA TRUMP': 0.669636181}, "Wrong result"

assert Organization['401 MEZZ VENTURE LLC']=={'TRUMP CHICAGO RETAIL LLC': 0.85298544}, "Wrong result"
assert Organization['ACE ENTERTAINMENT HOLDINGS INC']=={'TRUMP CASINOS INC.': 0.202484568,'TRUMP TAJ MAHAL INC.': 0.48784823299999996}, "Wrong result"
assert Organization['ANDREW JOBLON']=={'THE ERIC TRUMP FOUNDATION': 0.629688777}, "Wrong result"

print("\n(Passed.)")

,Entity B,Connection_Strength
Entity A,,
CARTER PAGE,DONALD J. TRUMP,0.694884
CHLOE MURDOCH,IVANKA TRUMP,0.805567
DONALD J. TRUMP,DONALD TRUMP JR.,0.453991
DONALD J. TRUMP,ERIC TRUMP,0.468002
DONALD J. TRUMP,IVANKA TRUMP,0.773875
DONALD J. TRUMP,MARYANNE TRUMP BARRY,0.330120
DONALD J. TRUMP,MELANIA TRUMP,0.517144
"DONALD J. TRUMP FOR PRESIDENT, INC.",DONALD J. TRUMP,0.377887
DONALD TRUMP JR.,ERIC TRUMP,0.405052


,Entity B,Connection_Strength
Entity A,,
401 MEZZ VENTURE LLC,TRUMP CHICAGO RETAIL LLC,0.852985
401 NORTH WABASH VENTURE LLC,TRUMP COMMERCIAL CHICAGO LLC,0.207837
401 NORTH WABASH VENTURE LLC,TRUMP PAYROLL CHICAGO LLC,0.910172
ABE WALLACH,"THE TRUMP ORGANIZATION, INC.",0.452901
ACE ENTERTAINMENT HOLDINGS INC,TRUMP CASINOS INC.,0.202485
ACE ENTERTAINMENT HOLDINGS INC,TRUMP TAJ MAHAL INC.,0.487848
AETOS SA,TRUMP MARKS PUNTA DEL ESTE LLC,0.516288
ALAN GARTEN,"THE TRUMP ORGANIZATION, INC.",0.569394
ALLEN WEISSELBERG,THE DONALD J. TRUMP FOUNDATION,0.968860



(Passed.)


**Exercise 2** (1 point). For the dictionary `Organization` **created in the above test cell**, create another dictionary `Organization_avg` which for every entity A gives the average connection strength (i.e., the average of nested dictionary values). `Organization_avg` should be in the following form:
```python
{Entity A: avg_Connection_Strength, Entity A: avg_Connection_Strength, ... }```


In [10]:
Organization_avg=dict()
for key in Organization:
    #print('key: ', key)
    count=0
    total=0
    for key2 in Organization[key]:
        #print('key2: ', key2)
        count+=1
        total+=Organization[key][key2]
    Organization_avg[key]=float(total/count)

In [11]:
# Test Cell: `Organization_avg`
assert type(Organization_avg)==dict or defaultdict, "Organization_avg is not a dictionary"
assert len(Organization_avg)==len(Organization)

for k_, v_ in {'401 MEZZ VENTURE LLC': 0.85298544,
               'DJT HOLDINGS LLC': 0.5855800477222223,
               'DONALD J. TRUMP': 0.4878277050144927,
               'JAMES BURNHAM': 0.187474088}.items():
    print(k_, Organization_avg[k_], v_)
    assert math.isclose(Organization_avg[k_], v_, rel_tol=4e-15*len(Organization[k_])), \
           "Wrong result for '{}': Expected {}, got {}".format(k_, v_, Organization_avg[k_])

print("\n(Passed.)")

401 MEZZ VENTURE LLC 0.85298544 0.85298544
DJT HOLDINGS LLC 0.5855800477222224 0.5855800477222223
DONALD J. TRUMP 0.4878277050144922 0.4878277050144927
JAMES BURNHAM 0.187474088 0.187474088

(Passed.)


**Exercise 3** (4 points). Based on the `Organization_avg` dictionary you just created, determine which organizations have an average connection strength that is strictly greater than a given threshold, `THRESHOLD` (defined in the code cell below). Then, create a new data frame named `Network_strong` that has a subset of the rows of `Network_sub` whose `Entity A` values match these organizations **and** whose `"Entity B Type"` equals `"Organization"`.

In [35]:
THRESHOLD = 0.5

# https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.from_dict.html
df=pd.DataFrame.from_dict(Organization_avg, orient='index')
df.columns=['conn_avg']

#display(Network_sub)

#Subset to get rows where avg>=threshold
df=df[df.conn_avg>THRESHOLD]
df.reset_index(inplace=True)
df2=df.merge(Network_sub, left_on='index', right_on='Entity A', how='inner')

#Enforcing 2nd condition
df3=df2[df2["Entity B Type"]=="Organization"]

#Drop unnecessary cols
df3.drop(['index','conn_avg'],axis=1,inplace=True)

#Rename column
#df3.rename({'conn_avg':'Connection_Strength'},axis=1,inplace=True)

print('df3.columns: ', df3.columns)

print(list(df3))

display(df3)


#Network_strong=df3[['Entity A Type', 'Entity A', 'Entity B Type', 'Entity B', 'Connection_Strength']]

Network_strong=df3

#Desired columns: 
#print(Network_sub.columns)



df3.columns:  Index(['Entity A Type', 'Entity A', 'Entity B Type', 'Entity B',
       'Connection_Strength'],
      dtype='object')
['Entity A Type', 'Entity A', 'Entity B Type', 'Entity B', 'Connection_Strength']


/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Entity A Type,Entity A,Entity B Type,Entity B,Connection_Strength
0,Organization,401 MEZZ VENTURE LLC,Organization,TRUMP CHICAGO RETAIL LLC,0.852985
1,Organization,401 NORTH WABASH VENTURE LLC,Organization,TRUMP COMMERCIAL CHICAGO LLC,0.207837
2,Organization,401 NORTH WABASH VENTURE LLC,Organization,TRUMP PAYROLL CHICAGO LLC,0.910172
3,Organization,AETOS SA,Organization,TRUMP MARKS PUNTA DEL ESTE LLC,0.516288
4,Person,ALAN GARTEN,Organization,"THE TRUMP ORGANIZATION, INC.",0.569394
5,Person,ALLEN WEISSELBERG,Organization,THE DONALD J. TRUMP FOUNDATION,0.968860
6,Person,ALLEN WEISSELBERG,Organization,THE DONALD J. TRUMP REVOCABLE TRUST,0.528962
7,Person,ALLEN WEISSELBERG,Organization,"THE TRUMP ORGANIZATION, INC.",0.580072
8,Person,ALLEN WEISSELBERG,Organization,TRUMP INTERNATIONAL GOLF CLUB LC,0.267505
9,Person,ALLEN WEISSELBERG,Organization,TRUMP INTERNATIONAL GOLF CLUB SCOTLAND LIMITED,0.800504


In [36]:
# Test Cell: `Network_strong`
assert type(Network_strong)==pd.DataFrame, "Network_strong is not a panda dataframe"
assert list(Network_strong)==['Entity A Type','Entity A','Entity B Type','Entity B','Connection_Strength'], "Your Network_strong columns are not consistent with the master dataset"
assert len(Network_strong)==194, "The length of your Network_strong is not correct. Correct length should be 194."
test2 = Network_strong.sort_values(by='Connection_Strength')
test2.reset_index(drop=True, inplace=True)
assert math.isclose(test2.loc[0, 'Connection_Strength'], 0.039889119, rel_tol=1e-13)
assert math.isclose(test2.loc[100, 'Connection_Strength'], 0.744171895, rel_tol=1e-13)
assert math.isclose(test2.loc[193, 'Connection_Strength'], 0.996641965, rel_tol=1e-13)

print("\n(Passed.)")


(Passed.)


#### **Fin!** Remember to test your solutions by running them as the autograder will: restart the kernel and run all cells from "top-to-bottom." Also remember to submit to the autograder; otherwise, you will **not** get credit for your hard work!